In [3]:
!pip install dspy

  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 2.9 MB/s  0:00:05m0:00:0100:01
Using cached mako-1.3.10-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [dspy]2m11/12 [dspy]lm]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import json
import dspy
import re
from pydantic import BaseModel, Field

In [2]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
class GameOf24(dspy.Signature):
    """
    Finalize the mathematical expression for the Game of 24.
    Given the original numbers and the successful reasoning steps, 
    format the final answer as a single mathematical expression.
    """
    problem = dspy.InputField(desc="The initial four numbers")
    reasoning_steps = dspy.InputField(desc="The sequence of valid moves found")
    expression = dspy.OutputField(desc="The final formula using parentheses, e.g., (4 * 3) * (1 + 1)")

In [4]:

# Enable DSPy internal logging
dspy.enable_logging()


class ProposeThoughts(dspy.Signature): 
    """
    Generate up to 10 distinct next steps. 
    EACH line must follow this format EXACTLY:
    <a> <op> <b> = <result> (Remaining: <list of all unused numbers and result>)
   
    Rules:
    1. Pick EXACTLY two numbers from the 'Available Numbers' list.
    2. Perform ONLY ONE operation.
    3. You MUST include the 'Remaining:' list on every line. If you start with [4, 9, 10, 13] and do 4+9=13, the 'Remaining' MUST be [10, 13, 13].
    4. CRITICAL: You MUST include a variety of operators (+, -, *, /) across the 10 steps.
    5.Output up to 10 distinct lines, no brackets, no lists, no explanations.
    """
    available_numbers = dspy.InputField(desc="The ONLY numbers currently available to use")
    target = dspy.InputField(desc="The goal number to reach")
    current_steps = dspy.InputField(desc="The history of moves and currently available numbers")
    proposals = dspy.OutputField(desc="up to 10 distinct lines of math")




# 1. Use Pydantic for structured scoring
class Evaluation(BaseModel):
    score: float = Field(description="A numeric score between 1 and 10")
    reasoning: str = Field(description="Brief explanation of the score")

class EvaluateThought(dspy.Signature):
    """
    Evaluate if a mathematical move brings us closer to the target.
    Score 10: The move hits the target exactly.
    Score 7-9: The move creates a number that is a factor or multiple of the target.
    Score 1: The move creates a large prime or number that makes the target impossible.
    OUTPUT ONLY the raw assessment data. 
    DO NOT include 'Score:' labels or conversational text.
    """
    target: str = dspy.InputField(desc="The number we are trying to reach")
    thought: str = dspy.InputField(desc="The specific math move being scored")
   
    assessment: Evaluation = dspy.OutputField(desc="Score and reasoning") # DSPy handles the parsing automatically

def validate_move(parsed, available):
    avail_copy = available.copy()
    try:
        avail_copy.remove(parsed["a"])
        avail_copy.remove(parsed["b"])
    except ValueError:
        # If the numbers aren't there, it's an invalid move
        return False, [] 
        
    ops = {'+': lambda x, y: x + y, '-': lambda x, y: x - y, '*': lambda x, y: x * y, '/': lambda x, y: x / y if y != 0 else None}
    
    python_result = ops[parsed['op']](parsed['a'], parsed['b'])
    
    # Strict Math Check: Does LLM result == Python result?
    if python_result is None or abs(python_result - parsed['result']) > 0.001:
        return False, [] # Math is wrong!
        
    # Compare with LM's claimed remaining
    # Calculate the new number and the new list
    new_result = int(parsed["result"])
    avail_copy.append(new_result)
    
    # Return (True, The actual new list)
    return True, sorted(avail_copy)

def parse_line(line: str):
    # 1. Standardise math symbols
    line = line.replace('x', '*').replace('×', '*')
    
    # 2. Flexible Regex: Handles optional parentheses around the equation
    # Pattern: Optional '(', num, op, num, Optional ')', '=', result, ... Remaining
    pattern = r"\(?(\d+)\s*([+\-*/])\s*(\d+)\)?\s*=\s*([\d\.]+).*?Remaining:?\s*([\d,\s\(\)\[\]]+)"
    
    match = re.search(pattern, line, re.IGNORECASE)
    if match:
        a, op, b, result, remaining_str = match.groups()
        
        # 3. Use findall to grab all digits from the 'Remaining' part
        clean_rem = [int(n) for n in re.findall(r"\d+", remaining_str)]
        
        return {
            "a": int(a),
            "b": int(b),
            "op": op,
            "result": float(result),
            "remaining": clean_rem
        }
    return None


# 2. Build the ToT Module
class TreeOfThoughts(dspy.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.proposer = dspy.ChainOfThought(ProposeThoughts)
         # Use standard Predict; it will honor the Type Hint in the Signature
        self.evaluator = dspy.Predict(EvaluateThought) 
        # ADD THIS: The final solver
        self.solver = dspy.Predict(GameOf24) 
        self.depth = depth

    def forward(self, problem,target):
        target_val = float(target)
        # Start with the initial problem state
        # SEED THE STATE: Make it impossible for the model to miss the numbers
        
        # Extract numbers from the problem string
        initial_nums = [int(x.strip()) for x in problem.split(",") if x.strip().isdigit()]
        
        candidates = [{"steps": f"Goal: {target}", "available": initial_nums, "score": 0.0}]

        for d in range(self.depth):
            print(f"depth is:+{d}")
            new_candidates = []
            for candidate in candidates:
                # Propose new branches
                # 1. Provide the LLM with the ONLY valid numbers for this branch
                res = self.proposer(
                    available_numbers=", ".join(map(str, candidate["available"])),
                    target=target,
                    current_steps=candidate["steps"]
                )
                
                # 2. Extract the proposals
                raw_proposals = res.proposals
                
                for line in raw_proposals.splitlines():
                    parsed = parse_line(line)
                    
                    if parsed:
                        # --- VALIDATION LOGIC ---
                        #  uses validate_move function
                        # Get the TRUTH from Python, not the LLM's hallucinated list
                        is_valid, actual_remaining = validate_move(parsed, candidate["available"])

                        if is_valid:
                            # --- SUCCESS CHECK ---
                            # If the result is 24, we don't need to score it or keep searching.
                            # We found a winner!

                            if abs(parsed["result"] - target_val) < 0.001 and len(actual_remaining) == 1:
                                # 1. Create a clean, verified string for the final move
                                clean_final_move = f"{parsed['a']} {parsed['op']} {parsed['b']} = {parsed['result']} (Remaining: {actual_remaining})"
                                print(f"🎯 TARGET FOUND: {clean_final_move}")
                                
                                # 2. Use the solver to get the final pretty equation (e.g., (4*3)*2)
                                final_formula = self.solver(problem=problem, reasoning_steps=candidate["steps"] + " -> " + clean_final_move)
                                
                                return dspy.Prediction(
                                    answer=final_formula.expression, 
                                    path=candidate["steps"] + " -> " + clean_final_move,
                                    score=10.0
                                )
                                
                            print(f"✅ Logic Pass: {parsed['a']} {parsed['op']} {parsed['b']} = {parsed['result']} (Remaining: {actual_remaining})")
                            
                            # 2. Evaluate the thought as before
                            print(f"Evaluating valid move: {parsed['a']} {parsed['op']} {parsed['b']} = {parsed['result']} (Remaining: {actual_remaining})")
                            eval_res = self.evaluator(target=target, thought=line)
                            
                            new_candidates.append({
                                "steps": candidate["steps"] + f" -> {parsed['a']} {parsed['op']} {parsed['b']} = {parsed['result']} (Remaining: {actual_remaining})",
                                "available": actual_remaining,  # truth from Python

                                "score": float(eval_res.assessment.score)
                            })
                        else:
                            print(f"Skipping illegal move (numbers not in list): {line}")

            # Selection step: Keep the top 4 best-scored paths (Beam Search)
            candidates = sorted(new_candidates, key=lambda x: x["score"], reverse=True)[:4]
            
        # After all depth levels are explored
        if not candidates:
            # Use 'path' and 'answer' so the print statement doesn't crash
            return dspy.Prediction(
                path="No valid math found", 
                answer="N/A", 
                score=0.0
            )
            

        # Use the Solver to wrap it all up
        best = candidates[0]
        if "24" in str(best["steps"]):
            # Use the solver to clean up the formatting of the successful path
            result = self.solver(problem=problem, reasoning_steps=best["steps"])
            return dspy.Prediction(answer=result.expression, path=best["steps"], score=10.0)
            
        result = self.solver(problem=problem, reasoning_steps=best["steps"])
        return dspy.Prediction(answer=result.expression, path=best["steps"], score=best["score"])
        

In [5]:
local_lm = dspy.LM('ollama_chat/'+MODEL, api_base='http://localhost:11434', api_key='')
local_lm.history = [] 
# Set it globally
#dspy.settings.configure(lm=local_lm)
dspy.settings.configure(lm=local_lm, cache=False) 

solver = TreeOfThoughts(depth=3)
problem_input = "1, 2, 3, 4"
result = solver(problem=problem_input,target=24)

print("Best Path:", result.path)
print("Equation:", result.answer)
print("Score:", result.score)


depth is:+0
✅ Logic Pass: 1 * 2 = 2.0 (Remaining: [2, 3, 4])
Evaluating valid move: 1 * 2 = 2.0 (Remaining: [2, 3, 4])
✅ Logic Pass: 1 + 2 = 3.0 (Remaining: [3, 3, 4])
Evaluating valid move: 1 + 2 = 3.0 (Remaining: [3, 3, 4])
✅ Logic Pass: 1 + 3 = 4.0 (Remaining: [2, 4, 4])
Evaluating valid move: 1 + 3 = 4.0 (Remaining: [2, 4, 4])
✅ Logic Pass: 2 + 3 = 5.0 (Remaining: [1, 4, 5])
Evaluating valid move: 2 + 3 = 5.0 (Remaining: [1, 4, 5])
✅ Logic Pass: 4 * 2 = 8.0 (Remaining: [1, 3, 8])
Evaluating valid move: 4 * 2 = 8.0 (Remaining: [1, 3, 8])
✅ Logic Pass: 1 * 4 = 4.0 (Remaining: [2, 3, 4])
Evaluating valid move: 1 * 4 = 4.0 (Remaining: [2, 3, 4])
✅ Logic Pass: 3 + 4 = 7.0 (Remaining: [1, 2, 7])
Evaluating valid move: 3 + 4 = 7.0 (Remaining: [1, 2, 7])
✅ Logic Pass: 2 * 3 = 6.0 (Remaining: [1, 4, 6])
Evaluating valid move: 2 * 3 = 6.0 (Remaining: [1, 4, 6])
Skipping illegal move (numbers not in list): 2+6=8 (Remaining: no 1 is available now and 4)
Skipping illegal move (numbers not in li

In [6]:
num_calls = len(local_lm.history)
print(f"Total prompt calls made: {num_calls}")

Total prompt calls made: 48


In [ ]:
dspy.inspect_history(n=49)